## Deliverable 2. Create a Customer Travel Destinations Map.

In [84]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [85]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Norman Wells,CA,65.2820,-126.8329,3.67,81,100,4.05,overcast clouds
1,1,Aklavik,CA,68.2191,-135.0107,-5.98,84,98,3.24,overcast clouds
2,2,Butaritari,KI,3.0707,172.7902,81.55,78,61,16.06,broken clouds
3,3,Lasa,IT,46.6166,10.7002,27.70,94,95,6.44,overcast clouds
4,4,Burnie,AU,-41.0667,145.9167,55.92,62,73,8.59,broken clouds


In [89]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature: '))
max_temp = float(input('What is the maximum temperature: '))

What is the minimum temperature: 75
What is the maximum temperature: 90


In [90]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) & (city_data_df['Max Temp'] >= min_temp)]

In [91]:
# 4a. Determine if there are any empty rows.
clean_df.count()

City_ID                216
City                   216
Country                216
Lat                    216
Lng                    216
Max Temp               216
Humidity               216
Cloudiness             216
Wind Speed             216
Current Description    216
dtype: int64

In [92]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = clean_df.dropna()

In [93]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Butaritari,KI,81.55,broken clouds,3.0707,172.7902,
5,Kahului,US,82.42,scattered clouds,20.8947,-156.4700,
12,Kapaa,US,78.78,overcast clouds,22.0752,-159.3190,
13,Kousseri,CM,86.34,broken clouds,12.0769,15.0306,
16,Anahuac,MX,81.81,overcast clouds,27.2333,-100.1500,
22,Nabire,ID,79.81,broken clouds,-3.3667,135.4833,
24,Puerto Ayora,EC,79.56,scattered clouds,-0.7393,-90.3518,
27,Makasar,ID,75.25,few clouds,-5.1400,119.4221,
36,Tabou,CI,78.33,light rain,4.4230,-7.3528,
37,Santa Rosa,PH,80.02,scattered clouds,14.3122,121.1114,


In [94]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        clean_hotel_df['Hotel Name'].replace("",np.nan, inplace= True)
        print('Hotel not found... skipping')
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [95]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = clean_hotel_df.dropna()

In [96]:
# 8a. Create the output File (CSV)
output_data_file = "../vacation_search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [103]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [104]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))